In [7]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D, Flatten

In [8]:
#Defining the Kerel and Bias Initializers
kernel_init = tf.keras.initializers.HeNormal
bias_init = tf.keras.initializers.constant(value=0.2)

def inception_module(x,
                     filter_1x1,
                     filter_3x3_reduce,
                     filter_3x3,
                     filter_5x5_reduce,
                     filter_5x5,
                     filter_pool,
                     name=None):
  #1x1 Convolution Block
  conv_1x1 = Conv2D(filter_1x1, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  #3x3 Convolution Block
  conv_3x3 = Conv2D(filter_3x3_reduce, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_3x3 = Conv2D(filter_3x3, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
  #5x5 Convolution Block
  conv_5x5 = Conv2D(filter_5x5_reduce, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_5x5 = Conv2D(filter_5x5, kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
  #Pooling Layer
  pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
  pool = Conv2D(filter_pool, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool)
  #Concatenate the blocks
  output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool], axis=3, name=name)

  return output

In [11]:
def InceptionNet(shape=(224,224,3), classes=10):
  #Input Layer
  input_layer = Input(shape=shape)
  x = Conv2D(62, kernel_size=(7,7), strides=2, padding='same', activation='relu', name='Conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
  x = MaxPooling2D((3,3), strides=2, padding='same', name='MaxPool_1_3x3/2')(x)
  x = Conv2D(192, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='Conv_2_3x3', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  x = MaxPooling2D((3,3), strides=2, padding='same', name='MaxPool_2_3x3/2')(x)
  #Inception Module 3a
  x = inception_module(x,
                       filter_1x1=64,
                       filter_3x3_reduce=96,
                       filter_3x3=128,
                       filter_5x5_reduce=16,
                       filter_5x5=32,
                       filter_pool=32,
                       name='inception_3a')
  #Inception Module 3b
  x = inception_module(x,
                       filter_1x1=128,
                       filter_3x3_reduce=128,
                       filter_3x3=192,
                       filter_5x5_reduce=32,
                       filter_5x5=96,
                       filter_pool=64,
                       name='inception_3b')
  x = MaxPooling2D((3,3), strides=2, padding='same', name='MaxPooling_3_3x3/2')(x)
  #Inception Module 4a
  x = inception_module(x,
                       filter_1x1=192,
                       filter_3x3_reduce=96,
                       filter_3x3=208,
                       filter_5x5_reduce=16,
                       filter_5x5=48,
                       filter_pool=64,
                       name='inception_4a')
  #1st Auxiliary Output
  x1 = AveragePooling2D((5,5), strides=3)(x)
  x1 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(x1)
  x1 = Flatten()(x1)
  x1 = Dense(1024, activation='relu')(x1)
  x1 = Dropout(0.7)(x1)
  x1 = Dense(classes, activation='softmax', name='Auxiliary_output_1')(x1)
  #Inception Module 4b
  x = inception_module(x,
                       filter_1x1=160,
                       filter_3x3_reduce=112,
                       filter_3x3=224,
                       filter_5x5_reduce=24,
                       filter_5x5=64,
                       filter_pool=64,
                       name='inception_4b')
  #Inception Module 4c
  x = inception_module(x,
                       filter_1x1=128,
                       filter_3x3_reduce=128,
                       filter_3x3=256,
                       filter_5x5_reduce=24,
                       filter_5x5=64,
                       filter_pool=64,
                       name='inception_4c')
  #Inception Module 4d
  x = inception_module(x,
                       filter_1x1=112,
                       filter_3x3_reduce=144,
                       filter_3x3=288,
                       filter_5x5_reduce=32,
                       filter_5x5=64,
                       filter_pool=64,
                       name='inception_4d')
  #2nd Auxiliary Output
  x2 = AveragePooling2D((5,5), strides=3)(x)
  x2 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(x2)
  x2 = Flatten()(x2)
  x2 = Dense(1024, activation='relu')(x2)
  x2 = Dropout(0.7)(x2)
  x2 = Dense(classes, activation='softmax', name='Auxiliary_Output_2')(x2)
  #Inception Module 4e
  x = inception_module(x,
                       filter_1x1=256,
                       filter_3x3_reduce=160,
                       filter_3x3=320,
                       filter_5x5_reduce=32,
                       filter_5x5=128,
                       filter_pool=128,
                       name='inception_4e')
  
  x = MaxPooling2D((3,3), strides=2, padding='same', name='MaxPooling_4_3x3/2')(x)
  #Inception Module 5a
  x = inception_module(x,
                       filter_1x1=256,
                       filter_3x3_reduce=160,
                       filter_3x3=320,
                       filter_5x5_reduce=32,
                       filter_5x5=128,
                       filter_pool=128,
                       name='inception_5a')
  #Inception Module 5b
  x = inception_module(x,
                       filter_1x1=384,
                       filter_3x3_reduce=192,
                       filter_3x3=384,
                       filter_5x5_reduce=48,
                       filter_5x5=128,
                       filter_pool=128,
                       name='inception_5b')
  x = AveragePooling2D((7,7), strides=1, padding='same', name='AveragePooling_1_7x7')(x)
  x = Dropout(0.4)(x)
  x = Dense(classes, activation='softmax', name='Output')(x)
  #Defining the Model
  model = tf.keras.models.Model(inputs=input_layer, outputs=x, name='InceptionNet')
  
  return model

In [13]:
model = InceptionNet(shape=(224,224,3), classes=10)
print(model.summary())

Model: "InceptionNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv_1_7x7/2 (Conv2D)          (None, 112, 112, 62  9176        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 MaxPool_1_3x3/2 (MaxPooling2D)  (None, 56, 56, 62)  0           ['Conv_1_7x7/2[0][0]']           
                                                                                       